<a href="https://colab.research.google.com/github/Will-est/PPO-From-Scratch/blob/main/PPO_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import statments
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import torch
import torch.nn as nn

ModuleNotFoundError: No module named 'stable_baselines3'

In [ ]:

# Hper parameters

initial_learning_rate = 0.1
env_id = "LunarLander"
clipping_coef = 0.1
num_envs = 4

# Conventional Vectoriz3ed Environment wrapper
def make_env(env_id):
    def _init():
        env = gym.make(env_id)
        # Optional: Add wrappers here if needed
        return env
    return _init

# Initialize info dictionary or maybe a list of dictionaries where each entry contains the mean reward, loss, number of steps, learning rate
info = []

# Agent definition

class Agent(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
      super().__init__(*args, **kwargs):
      self.agent = nn.Sequential([])
      self.agent = nn.Sequential([])
  def predict():


if __name__ == "main":


  # initilize invironment

  # put things onto gpu

  # define training regime

  # outer loop: rollouts, at each iteration of the larger loop do one step for your policy and update your value function make sure to clip both the policy and value function. ALso include an entropy term

  # inner loop: train value function, step through environment, record rewards, ends, log probs, etc.

